## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import requests as req
from pprint import pprint as p

# Import API key
from config import g_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_api_key)

# Base URL
url_1 = 'https://www.google.com/maps/dir/?api=' + g_api_key
url_2 = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Resources/WeatherPy_database.csv")

# Check imported data set
city_data_df.head()
print(len(city_data_df.index))

696


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = input("Minimum Temperatur: ")
max_temp = input("Maximum Temperatur: ")


In [18]:
# convert to integer
min_temp = int(min_temp)
max_temp = int(max_temp)

In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_city_data_df = city_data_df.loc[(city_data_df['Max Temp']>= min_temp) & (city_data_df['Max Temp']<= max_temp)]

filtered_city_data_df
print(len(filtered_city_data_df.index))

406


In [20]:
# 4a. Determine if there are any empty rows.
filtered_city_data_df.count()

City_ID                406
City                   406
Lat                    406
Lng                    406
Max Temp               406
Current Description    406
Rain (inches)          406
Snow (inches)          406
Humidity               406
Cloudiness             406
Wind Speed             406
Country                403
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filtered_city_data_df = filtered_city_data_df.dropna()


In [22]:
print(len(filtered_city_data_df.index))

403


In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = []
hotel_df = filtered_city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

x = [i for i in range(len(hotel_df.index))]

# 5b. Create a new column "Hotel Name"
hotel_df['Hotel Name'] = ""
hotel_df['index_'] = x

hotel_df = hotel_df.set_index('index_')
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
index_,,,,,,,
0,Belmonte,BR,77.50,moderate rain,-15.8631,-38.8828,
1,Saint George,US,50.47,clear sky,37.1041,-113.5841,
2,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
3,Kathmandu,NP,46.62,few clouds,27.7167,85.3167,
4,Busselton,AU,72.09,clear sky,-33.6500,115.3333,
5,Rikitea,PF,77.07,overcast clouds,-23.1203,-134.9692,
6,Pangkalanbuun,ID,76.55,overcast clouds,-2.6833,111.6167,
7,Taoudenni,ML,80.53,overcast clouds,22.6783,-3.9836,
8,Ushuaia,AR,58.66,scattered clouds,-54.8000,-68.3000,


In [31]:
print(len(filtered_city_data_df.index))

403


In [32]:
#df.loc[(df['Salary_in_1000']>=100) & (df['Age']< 60) & (df['FT_Team'].str.startswith('S')),['Name','FT_Team']]
lat__ = hotel_df['Lat']
lng__ = hotel_df['Lng']

#lat__.iloc[0]
#lng__.iloc[0]
#
#print (lat__)
#print (lng__)


In [27]:
# 6a. Set parameters to search for hotels within 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_api_key
}


# 6b. Iterate through the hotel DataFrame.
for i, city in enumerate(hotel_df):
    # 6c. Get latitude and longitude from DataFrame
    lat_i = str(lat__.iloc[i]) 
    lng_i = str(lng__.iloc[i])


    # creating location key on 1st pass // replacing lat&long string on subsequent passes
    params["location"] = lat_i + ',' + lng_i
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # this is above // url_2
    #print (params)
    
    # 6e. Make request and retrieve the JSON data from the search. 
    try:
        city_hotel = req.get(url_2, params = params).json()
        p(city_hotel)

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        #df.loc[(df['Salary_in_1000']>=100) & (df['Age']< 60) & (df['FT_Team'].str.startswith('S')),['Name','FT_Team']]    

        hotel_name = city_hotel['results'][0]['name']
        hotel_df.loc[index_,['Hotel Name']]= hotel_name 

        #hotel_df.loc[index, 'Hotel Name'] = city_hotel['results'][0]['name']

    except:
        print("Hotel not found in City. Skipping...")
        pass     
        

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -15.8590807, 'lng': -38.8788983},
                           'viewport': {'northeast': {'lat': -15.8577392197085,
                                                      'lng': -38.87753036970851},
                                        'southwest': {'lat': -15.8604371802915,
                                                      'lng': -38.8802283302915}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Pousada Monte Carmelo O Pão',
              'photos': [{'height': 2338,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/contrib/108836040145173663368">M

In [28]:
print(len(hotel_df.index))

404


In [29]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
index_,,,,,,,
0,Belmonte,BR,77.50,moderate rain,-15.8631,-38.8828,
1,Saint George,US,50.47,clear sky,37.1041,-113.5841,
2,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
3,Kathmandu,NP,46.62,few clouds,27.7167,85.3167,
4,Busselton,AU,72.09,clear sky,-33.6500,115.3333,
...,...,...,...,...,...,...,...
399,Ati,TD,81.73,scattered clouds,13.2154,18.3353,
400,Mumbwa,ZM,69.28,moderate rain,-14.9783,27.0619,
401,Vaitape,PF,80.04,few clouds,-16.5167,-151.7500,


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] = '']


In [ ]:
hotel_df

In [ ]:
# 8a. Create the output File (CSV)


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
